# Project - Airline AI Assistant

We'll now bring together what we've learned to make an AI Customer Support assistant for an Airline

In [0]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
from IPython.display import display, JSON

In [0]:
# Initialization

load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

# As an alternative, if you'd like to use Ollama instead of OpenAI
# Check that Ollama is running for you locally (see week1/day2 exercise) then uncomment these next 2 lines
# MODEL = "llama3.2"
# openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')


In [0]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [0]:
# This function looks rather simpler than the one from my video, because we're taking advantage of the latest Gradio updates

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

## Tools

Tools are an incredibly powerful feature provided by the frontier LLMs.

With tools, you can write a function, and have the LLM call that function as part of its response.

Sounds almost spooky.. we're giving it the power to run code on our machine?

Well, kinda.

In [0]:
# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

def book_ticket(destination_city, price):
    print(f"Tool book_ticket for {destination_city} for {price}")
    list_price = get_ticket_price(destination_city)
    if list_price != "Unknown":
        list_amount = int(list_price.replace("$", ""))
        amount = int(price.replace("$", ""))
        if list_amount > amount:
            return "Booking Successful at a Discount!"
        else:
            return "Booking Successful"
    else:       
        return "Booking Failed: reason was that no list price was found for this destination"

In [0]:
book_ticket("Berliner", "$388")

In [0]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

book_function = {
    "name": "book_ticket",
    "description": """Get the success status of a function that can book a ticket using a city and a price. 
    Call this whenever you are asked to book a ticket, 
    for example when a customer asks 'Please can I book a ticket to Paris' or after you have asked 
    if they would like to book a ticket, for example, after you have supplied a ticket price. 
    If the customer negotiates and asks for a discount, use the agreed price, otherwise use the price that 
    matches the destination city. 
    It is really important that you confirm that the customer is happy to proceed with an agreed 
    booking after reading back the destination city and the agreed price.""",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
            "price": {
                "type": "string",
                "description": "The price that the customer has agreed to pay for the ticket",
            },
        },
        "required": ["destination_city", "price"],
        "additionalProperties": False
    } 
}

In [0]:
# And this is included in a list of tools:

tools = [
    {"type": "function", "function": price_function},
    {"type": "function", "function": book_function}
]

## Getting OpenAI to use our Tool

There's some fiddly stuff to allow OpenAI "to call our tool"

What we actually do is give the LLM the opportunity to inform us that it wants us to run the tool.

Here's how the new chat function looks:

In [0]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    
    display(JSON(messages))
    display(response)
    
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        messages.append(message)
        messages.extend(list(map(handle_tool_call, message.tool_calls)))
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [0]:
# We have to write that function handle_tool_call:

def handle_tool_call(tool_call):
    function = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)
    match function:
        case 'get_ticket_price':
            city = arguments.get('destination_city')
            price = get_ticket_price(city)
            return {
                "role": "tool",
                "content": json.dumps({"destination_city": city,"price": price}),
                "tool_call_id": tool_call.id
            }
        case 'book_ticket':
            city = arguments.get('destination_city')
            price = arguments.get('price')
            status = book_ticket(city, price)
            return {
                "role": "tool",
                "content": json.dumps({"destination_city": city,"price": price, "status": status}),
                "tool_call_id": tool_call.id
            }
    

In [0]:
gr.ChatInterface(fn=chat, type="messages").launch()

It can be really fun to book at a different price. Sometimes the LLM can correctly tell you the amount of money you saved. This could easily be expanded to haggle with a lower limit.